# Name : Samer AlTaki
### IOT & Computer Vision 
## Task3 : Social Distance Detector in Real Time

#   

# 1.1 Social Distance Detector

In [12]:
import numpy as np
import time
import cv2
import math

In [13]:
#Load the COCO class labels:
classes = []
with open("yolov3/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

#Load the pre-trained YOLOv3 weights and configuration files for object detection
net = cv2.dnn.readNetFromDarknet("yolov3/yolov3.cfg" , "yolov3/yolov3.weights")

#Define the minimum distance threshold for social distancing (in pixels) and set up a scale factor for resizing the image
min_distance = 120
scale = 0.00392

In [14]:
cap = cv2.VideoCapture('test_video.mp4')
hasFrame, frame = cap.read()

In [15]:
while True:
    ret, image = cap.read()
    if not ret:
        break
    height, width, _ = image.shape
    # Perform object detection using YOLOv3
    blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Initialize lists for storing person centroids and bounding boxes
    class_ids = []
    confidences = []
    boxes = []
    centroids = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if class_id == 0 and confidence > 0.5:
                # Object is a person
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calculate the bounding box coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Store the person's centroid and bounding box
                centroids.append((center_x, center_y))
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Perform non-maximum suppression to eliminate redundant overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Check the distance between each pair of centroids
    for i in range(len(centroids)):
        if i in indices:
            x1, y1 = centroids[i]
            for j in range(i + 1, len(centroids)):
                if j in indices:
                    x2, y2 = centroids[j]
                    distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

                    # If distance is less than the threshold, draw a red bounding box
                    if distance < min_distance:
                        cv2.rectangle(image, (boxes[i][0], boxes[i][1]), (boxes[i][0] + boxes[i][2], boxes[i][1] + boxes[i][3]), (0, 0, 255), 2)
                        cv2.rectangle(image, (boxes[j][0], boxes[j][1]), (boxes[j][0] + boxes[j][2], boxes[j][1] + boxes[j][3]), (0, 0, 255), 2)
                    # If distance is larger than the threshold, draw a green bounding box
                    else:
                        cv2.rectangle(image, (boxes[i][0], boxes[i][1]), (boxes[i][0] + boxes[i][2], boxes[i][1] + boxes[i][3]), (0, 255, 0), 2)
                        cv2.rectangle(image, (boxes[j][0], boxes[j][1]), (boxes[j][0] + boxes[j][2], boxes[j][1] + boxes[j][3]), (0, 255, 0), 2)

    # Display the resulting image
    cv2.imshow("Social Distancing Detector", image)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()